In [1]:
import sys
sys.path.append('..')
import numpy as np
from scipy.optimize._numdiff import approx_derivative

from dynamics import LowThrustTwoBody
from units import *

%load_ext autoreload
%autoreload 2

In [2]:
MU = 132712440018  # km^3/s^2
g0 = 9.8065e-3  # km/s^2
m0 = 4000  # kg
Tmax = 0.32e-3  # kN
Isp = 3000  # s

rv0 = np.array([
    -3637871.081, 147099798.784, -2261.441,
    -30.265097, -0.8486854, 0.0000505
])  # [km] and [km/s]

# t0 = 56284 * 86400  # MJD * sec
t0 = 0  # MJD * sec
dt = 3534 * 86400 / TU
tn = t0 + dt

# normalize
rv0[:3] /= DU
rv0[3:] /= (DU / TU)
m0 /= MaU
MU /= (DU ** 3 / TU ** 2)
Tmax /= (MaU * DU / TU ** 2)
Isp /= TU
g0 /= (DU / TU ** 2)

np.random.seed(0)
costate = np.random.rand(7) * 0.1
x0 = np.r_[rv0, m0, costate]
print(x0)

[-2.43176662e-02  9.83301421e-01 -1.51167994e-05 -1.01611007e+00
 -2.84934750e-02  1.69546982e-06  1.00000000e+00  5.48813504e-02
  7.15189366e-02  6.02763376e-02  5.44883183e-02  4.23654799e-02
  6.45894113e-02  4.37587211e-02]


In [3]:
dyn = LowThrustTwoBody(
    mu=MU, thrust_max=Tmax, Isp=Isp, g0=g0, rho=1 
)
dyn.variation = False
print(dyn.time_derivative(t0, x0).shape)

(14,)


In [4]:
def F(X):
    return dyn.time_derivative(t0, X)

In [5]:
dFdX_num = approx_derivative(F, x0)

In [6]:
Phi0 = np.eye(14)
X0 = np.r_[rv0, m0, costate, Phi0.ravel()]

dyn2 = LowThrustTwoBody(
    mu=MU, thrust_max=Tmax, Isp=Isp, g0=g0, rho=1 
)
dyn2.variation = True
print(dyn2.time_derivative(t0, X0).shape)

(210,)


In [7]:
dFdX_theo = dyn2.dFdX.jac_arr

In [8]:
with np.printoptions(precision=1):
    print(dFdX_num)

[[ 0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [-1.0e+00 -7.8e-02  1.2e-06  0.0e+00  0.0e+00  0.0e+00  1.4e-03  0.0e+00
   0.0e+00  0.0e+00 -1.6e-02  4.7e-03  7.1e-03 -2.0e-03]
 [-7.8e-02  2.1e+00 -4.8e-05  0.0e+00  0.0e+00  0.0e+00  1.1e-03  0.0e+00
   0.0e+00  0.0e+00  4.7e-03 -1.8e-02  5.6e-03 -1.6e-03]
 [ 1.2e-06 -4.8e-05 -1.1e+00  0.0e+00  0.0e+00  0.0e+00  1.6e-03  0.0e+00
   0.0e+00  0.0e+00  7.1e-03  5.6e-03 -1.3e-02 -2.4e-03]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  3.3e-04  0.0e+00
   0.0e+00  0.0e+00 -2.0e-03 -1.6e-03 -2.4e-03 -3.5e-03]
 [-1.2e-01 -1.9e-01  5.1e-03  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+0

In [9]:
with np.printoptions(precision=1):
    print(dFdX_theo)

[[ 0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [-1.0e+00 -7.8e-02  1.2e-06  0.0e+00  0.0e+00  0.0e+00  1.4e-03  0.0e+00
   0.0e+00  0.0e+00 -1.6e-02  4.7e-03  7.1e-03 -2.0e-03]
 [-7.8e-02  2.1e+00 -4.8e-05  0.0e+00  0.0e+00  0.0e+00  1.1e-03  0.0e+00
   0.0e+00  0.0e+00  4.7e-03 -1.8e-02  5.6e-03 -1.6e-03]
 [ 1.2e-06 -4.8e-05 -1.1e+00  0.0e+00  0.0e+00  0.0e+00  1.6e-03  0.0e+00
   0.0e+00  0.0e+00  7.1e-03  5.6e-03 -1.3e-02 -2.4e-03]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  3.3e-04  0.0e+00
   0.0e+00  0.0e+00 -2.0e-03 -1.6e-03 -2.4e-03 -3.5e-03]
 [-1.2e-01 -1.9e-01  5.1e-03  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+0

In [10]:
with np.printoptions(precision=1):
    print(dFdX_num - dFdX_theo)

[[ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 5.9e-11 -9.4e-12  1.5e-12  0.0e+00  0.0e+00  0.0e+00 -1.9e-13  0.0e+00
   0.0e+00  0.0e+00 -1.7e-11 -1.9e-11 -9.9e-12  7.7e-14]
 [ 2.1e-11  1.3e-10 -5.8e-11  0.0e+00  0.0e+00  0.0e+00 -1.1e-11  0.0e+00
   0.0e+00  0.0e+00 -9.7e-12  1.3e-11 -4.4e-12  8.2e-13]
 [ 2.5e-15 -2.1e-14  6.0e-11  0.0e+00  0.0e+00  0.0e+00  8.2e-15  0.0e+00
   0.0e+00  0.0e+00 -2.0e-11 -2.3e-11 -2.7e-11  6.4e-14]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -6.0e-14  0.0e+00
   0.0e+00  0.0e+00  2.4e-12  2.1e-12  2.2e-12  1.2e-13]
 [ 1.0e-11 -2.3e-11  4.3e-12  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+0

In [11]:
print(np.max(abs(dFdX_num - dFdX_theo)))
max_idx = np.argmax(abs(dFdX_num - dFdX_theo))
print(np.unravel_index(max_idx, dFdX_num.shape))

1.318563036534215e-10
(4, 1)


In [12]:
%%timeit
dyn.time_derivative(t0, x0)
# 35.6 µs ± 497 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

31.3 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
%%timeit
dyn2.time_derivative(t0, X0)
# 440 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

430 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
